In [1]:
%pip install 'zenml[server]'

Note: you may need to restart the kernel to use updated packages.


In [2]:
!zenml integration install sklearn -y

⠇ Installing integrations.....


In [3]:
%pip install pyparsing==2.4.2

Note: you may need to restart the kernel to use updated packages.


In [4]:
import IPython

In [5]:
# IPython.Application.instance().kernel.do_shutdown(restart=True)

In [6]:
NGROK_TOKEN = "" # todo: setting  ngrok token if colab

In [7]:
%pip install zenml

Note: you may need to restart the kernel to use updated packages.


In [8]:
# %pip install zenml  # Install zenml
from zenml.environment import Environment # Corrected import statement
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtokent{NGROK_TOKEN}

In [9]:
!rm -rf . zen
!zenml init

rm: refusing to remove '.' or '..' directory: skipping '.'


Found existing ZenML repository at path '/home/ry9/Downloads'.
⠋ Initializing ZenML repository at /home/ry9/Downloads.
⠋ Initializing ZenML repository at /home/ry9/Downloads.



In [10]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
def train_test()->None:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
  classifier = SVC(gamma=0.001)
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy}")

train_test()

Accuracy: 0.9583333333333334


# Turning Experiments into ML Pipelines with ZenML

In [12]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install typing_extensions

Note: you may need to restart the kernel to use updated packages.


In [14]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

In [15]:
@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
  return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> SVC:
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: SVC,
)-> float:
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc



In [16]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train , X_test, y_train, y_test = importer()
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test=X_test, y_test=y_test, model=model)


# Running Zenml Pipelines

In [17]:
%pip install dask[dataframe]

zsh:1: no matches found: dask[dataframe]
Note: you may need to restart the kernel to use updated packages.


In [18]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new pipeline: digits_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
No materializer is registered for type <class 'numpy.ndarray'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'numpy.ndarray'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
No materializer is registered for type <class 'numpy.ndarray'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loa

In [20]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"Dashboard running at {public_url}")
    !zenml up --blocking --port {port}
  else:
    !zenml up --port {port}

start_zenml_dashboard()

The `zenml up` command is deprecated and will be removed in a future release. 
Please use the `zenml login --local` command instead.
Calling `zenml login --local`...
Deploying a local daemon ZenML server.
⠋ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠙ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠹ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠸ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠼ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠴ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠦ Starting service 'DaemonZenServer[2a0ebc80-a921-40e0-8f6e-b82e6ab5383d] (type:
zen_server, flavor: daemon)'.
⠧ Starting service